In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="quick-starts-ws-121429")
exp = Experiment(workspace=ws, name="quick-starts-ws-121429")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-121429
Azure region: southcentralus
Subscription id: 888519c8-2387-461a-aff3-b31b86e2438e
Resource group: aml-quickstarts-121429


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

from azureml.core.compute_target import ComputeTargetException

# Choose a name for your CPU cluster
cpu_cluster_name = "cluster-notebook"

# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

Creating
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [5]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive import choice
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform
import os

# Specify parameter sampler
ps = RandomParameterSampling({
    '--C': choice(0.1, 0.3, 0.5, 0.7, 1.0),
    '--max_iter': choice(100, 150, 200, 250, 300)
})

# Specify a Policy
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
est = SKLearn(source_directory="./", compute_target=cpu_cluster, entry_script="train.py")

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(estimator=est,
                                hyperparameter_sampling=ps,
                                policy=policy,
                                primary_metric_name='Accuracy',
                                primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                max_total_runs=20,
                                max_concurrent_runs=4)

In [6]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

hdr = exp.submit(config=hyperdrive_config)
RunDetails(hdr).show()

WARNING - If 'script' has been provided here and a script file name has been specified in 'run_config', 'script' provided in ScriptRunConfig initialization will take precedence.


_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [17]:
import joblib
# Get your best run and save the model from that run.

best_model = hdr.get_best_run_by_primary_metric()

joblib.dump(best_model, "Best_Model.joblib")

TypeError: can't pickle _thread.RLock objects

In [10]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

path = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
ds = TabularDatasetFactory.from_delimited_files(path)

In [13]:
import pandas as pd
from train import clean_data
from sklearn.model_selection import train_test_split

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33)

training_data = pd.concat([x_train, y_train], axis=1)
training_data.head()

,age,marital,default,housing,loan,month,day_of_week,duration,campaign,pdays,...,contact_telephone,education_basic.4y,education_basic.6y,education_basic.9y,education_high.school,education_illiterate,education_professional.course,education_university.degree,education_unknown,y
25649,54,1,0,0,0,7,2,604,2,999,...,0,0,0,0,0,0,1,0,0,0
9986,25,0,0,0,0,7,4,85,3,999,...,0,0,0,0,1,0,0,0,0,0
1598,33,1,0,0,0,5,5,89,6,999,...,0,0,0,0,0,0,0,1,0,0
15393,32,0,0,0,1,3,4,167,1,999,...,0,0,0,0,0,0,0,1,0,1
30838,51,1,0,0,0,8,3,90,1,999,...,0,0,0,1,0,0,0,0,0,0


In [14]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='Accuracy',
    training_data=training_data,
    label_column_name='y',
    n_cross_validations=5)

In [16]:
# Submit your automl run

remote_run = exp.submit(config=automl_config, show_output=False)

ConfigException: ConfigException:
	Message: Invalid argument(s) 'primary_metric' specified. Supported value(s): 'accuracy, precision_score_weighted, average_precision_score_weighted, AUC_weighted, norm_macro_recall'.
	InnerException: None
	ErrorResponse 
{
    "error": {
        "code": "UserError",
        "message": "Invalid argument(s) 'primary_metric' specified. Supported value(s): 'accuracy, precision_score_weighted, average_precision_score_weighted, AUC_weighted, norm_macro_recall'.",
        "details_uri": "https://aka.ms/AutoMLConfig",
        "target": "primary_metric",
        "inner_error": {
            "code": "BadArgument",
            "inner_error": {
                "code": "ArgumentInvalid"
            }
        }
    }
}

In [ ]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###
best_automl_model = remote_run.get_best_run_by_primary_metric()